# How to create a deep learning classifier to detect people with no mask for COVID-19 in a crowd of people

![](sample_images/cover.png)

## About the author: 

[Merishna S](https://www.linkedin.com/in/merishna-ss/) is an Artificial intelligence & Machine Learning engineer with strong fundamentals in machine learning algorithms (neural networks, dimensionality reduction, feature utilization, and extraction and clustering), programming, statistics, and mathematics. 

## Introduction

The [CDC](https://www.cdc.gov/) continues to monitor the spread of COVID-19 and advises people who are completely vaccinated as well as those who are not fully vaccinated to wear face masks. When visiting the doctor's office, hospitals, or long-term care institutions, the CDC recommends wearing masks and keeping a safe distance. 

Manually monitoring people entering such institutions is tedious and requires workforce. In this tutorial, we will learn how we can automate this process through deep learning techniques which will automatically detect people not wearing masks to prevent their entry.

## Glossary

**Deep Learning:** It is a kind of machine learning technique that enables learning through the use of neural networks that mimic the human brain.

## Prerequisites

1. Programming knowledge in Python.
2. Basic knowledge of Jupyter Notebook, Deep Learning, Keras.

## Creating the mask detection deep learning model

We will now look into building a Deep Learning model to predict (detect) if a person is violating the rules by not wearing a mask in public spaces.

### Importing Python libraries

In [16]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
from scipy.spatial import distance
import matplotlib.pyplot as plt
import os

from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import Sequential
from keras.layers import Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

### Step 1: Getting the data

For the training data, we are using the face mask detection data from [here](https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset). The dataset contains 12 thousand images divided into Test, Train, and Validation sets which were scraped from Google and the CelebFace dataset created by [Jessica Li ](https://www.kaggle.com/jessicali9530). To start using it, you can download the dataset and save it in the working directory.

In [ ]:
# Load train and test set
train_dir = "data/Train"
test_dir = "data/Test"
val_dir = "data/Validation"

### Step 2: Reading a sample image and performing face detection

We will now read in a sample image from a busy airport and perform face detection using haar cascade classifier. The [Haar cascade classifier](https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html), originally known as the Viola-Jones Face Detection Technique is a object detection algorithm for detecting faces in images or real-time video. 

Viola and Jones proposed edge or line detection features in their research paper "Rapid Object Detection using a Boosted Cascade of Simple Features," published in 2001. The algorithm is given a large number of positive photos with faces and a large number of negative images with no faces. The model developed as a result of this training can be found in the OpenCV GitHub [repository](https://github.com/opencv/opencv/tree/master/data/haarcascades).

In [10]:
# Read a sample image
img = cv2.imread("sample_images/image (24)")

# Keep a copy of coloured image
orig_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image

# Convert image to grayscale
img = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)

# loading haarcascade_frontalface_default.xml
face_detection_model = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")

# detect faces in the given image
return_faces = face_detection_model.detectMultiScale(
    img, scaleFactor=1.1, minNeighbors=4
)  # returns a list of (x,y,w,h) tuples

# plotting the returned values
for (x, y, w, h) in return_faces:
    cv2.rectangle(orig_img, (x, y), (x + w, y + h), (0, 0, 255), 1)

plt.figure(figsize=(12, 12))
plt.imshow(orig_img)  # display the image

### Step 3: Detecting social distancing between people

We will now attempt to add a social distancing violation detector between people based on the distance between the coordinates of their faces. If the distance is less than the minimum distance, then they are not following social distancing norms (shown by red bounding box).

In [14]:
# minimum distance between two people (approx. 6ft)
MIN_SOCIAL_DISTANCE = 130

# check if the euclidean distance between the coordinates of faces is less than minimum distance
if len(return_faces) >= 2:
    label = [0 for i in range(len(return_faces))]
    for i in range(len(return_faces) - 1):
        for j in range(i + 1, len(return_faces)):
            dist = distance.euclidean(return_faces[i][:2], return_faces[j][:2])
            # if the distance is less
            if dist < MIN_SOCIAL_DISTANCE:
                label[i] = 1
                label[j] = 1
    new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image
    for i in range(len(return_faces)):
        (x, y, w, h) = return_faces[i]
        if label[i] == 1:
            cv2.rectangle(
                new_img, (x, y), (x + w, y + h), (255, 0, 0), 1
            )  # red bounding box (unsafe)
        else:
            cv2.rectangle(
                new_img, (x, y), (x + w, y + h), (0, 255, 0), 1
            )  # green bounding box (safe)
    plt.figure(figsize=(10, 10))
    plt.imshow(new_img)

else:
    print("No. of faces detected is less than 2")

### Step 4: Data preprocessing for building the mask detection Keras model

We will now pass our datasets into Keras ImageDataGenerator() to perform some preliminary data augmentation steps such as rescaling. 

In [18]:
# Data preprocessing
# Train
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2
)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)

# Validation
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_generator = train_datagen.flow_from_directory(
    directory=val_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)

# Test
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = train_datagen.flow_from_directory(
    directory=val_dir, target_size=(128, 128), class_mode="categorical", batch_size=32
)

### Step 5: Create the mask detection transfer learning model using Keras

We are building the deep learning classifer using the VGG19 transfer learning model. The VGG19 model is the successor of AlexNet, a variation of the VGG model named after the group named as Visual Geometry Group at Oxford which created it. It is a deep CNN consisting of 19 layers (16 convolution layers, 3 Fully connected layer, 5 MaxPool layers and 1 SoftMax layer) used to classify images. 

It has been trained on [ImageNet](https://image-net.org/), a picture database with 14,197,122 images structured according to the WordNet hierarchy.

In [20]:
vgg19_model = VGG19(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

for layer in vgg19_model.layers:
    layer.trainable = False

# Initialize a sequential model
model = Sequential()
model.add(vgg19_model)
model.add(Flatten())
model.add(Dense(2, activation="sigmoid"))
model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="accuracy")

### Step 6: Train the model

In [ ]:
# Fit the model on train data along with validation data
model_history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator) // 32,
    epochs=20,
    validation_data=val_generator,
    validation_steps=len(val_generator) // 32,
)

# Evaluate model performance on test data
model.evaluate_generator(test_generator)

### Step 7: Save the model

In [ ]:
model.save('saved_model.h5')

## Step 7: Test the model on the sample image

We will now test the trained model on our use case for detecting faces and masks for a group of people. We take the detected face crops of the faces detected in the image and then predict the mask or no mask using the model trained.

In [ ]:
# label for mask detection
mask_det_label = {0: "MASK", 1: "NO MASK"}
# label for social distancing norm
social_dist_label = {0: (0, 255, 0), 1: (255, 0, 0)}

# If the image contains more than 2 faces
if len(return_faces) >= 2:
    label = [0 for i in range(len(return_faces))]
    for i in range(len(return_faces) - 1):
        for j in range(i + 1, len(return_faces)):
            # Check the euclidean distance between
            dist_between = distance.euclidean(return_faces[i][:2], return_faces[j][:2])
            if dist_between < MIN_SOCIAL_DISTANCE:
                label[i] = 1
                label[j] = 1

    main_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image
    for i in range(len(return_faces)):
        (x, y, w, h) = return_faces[i]
        cropped_face = main_img[y : y + h, x : x + w]
        cropped_face = cv2.resize(cropped_face, (128, 128))
        cropped_face = np.reshape(cropped_face, [1, 128, 128, 3]) / 255.0
        mask_result = model.predict(cropped_face)
        cv2.putText(
            main_img,
            mask_det_label[mask_result.argmax()],
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            social_dist_label[label[i]],
            2,
        )
        cv2.rectangle(main_img, (x, y), (x + w, y + h), social_dist_label[label[i]], 1)
    plt.figure(figsize=(10, 10))
    plt.imshow(main_img)

else:
    print("No. of faces detected is less than 2")